We use math to coherently understand the world. One of the most useful branches of math for working with data, and where our computers can be extra helpful, is linear algebra.

## Space, the first frontier

The DSFS book and most other treatments of linear algebra begin with *vectors*. We will get there. But vectors have to exist somewhere. So I think we should start with a *vector space*. This is a representation where we can identify specific vectors. Just think about this as some arbitrary space. It has directions (like an x axis, and a y-axis, and a z-axis, and even if you want a kangaroo axis).

Let's draw a two dimensional space (2-D) to get started. 

In [594]:
import altair as alt
import numpy as np
import pandas as pd
from typing import List, Tuple

In [595]:
my_vecs = pd.DataFrame({'x': [0], 'y': [0]})

In [596]:
alt.Chart(my_vecs).mark_trail().encode(
    x='x', 
    y='y')

alt.Chart(...)

Every point in this space is defined by two numbers, a x-coordinate and a y-coordinate.


In [597]:
my_valuesX = np.linspace(-5, 5, num=51)
my_valuesY = np.linspace(-5, 5, num=51)
X, Y = [i.ravel() for i in np.meshgrid(my_valuesX, my_valuesY)]

In [598]:
X,Y

(array([-5. , -4.8, -4.6, ...,  4.6,  4.8,  5. ]),
 array([-5., -5., -5., ...,  5.,  5.,  5.]))

In [599]:
my_grid = pd.DataFrame()
my_grid['x'] = X
my_grid['y'] = Y
my_grid

x    y
0    -5.0 -5.0
1    -4.8 -5.0
2    -4.6 -5.0
3    -4.4 -5.0
4    -4.2 -5.0
...   ...  ...
2596  4.2  5.0
2597  4.4  5.0
2598  4.6  5.0
2599  4.8  5.0
2600  5.0  5.0

[2601 rows x 2 columns]

In [600]:
base = alt.Chart(my_grid)

pointsIn2D = base.mark_point(opacity=.25).encode(
    x='x:Q', 
    y='y:Q')
pointsIn2D

alt.Chart(...)

So the space is just *room* for a specific type of stuff. Here the stuff is pairs of x and y values. Since this is 2-space, it is defined across an area.

But even in this simple case, we are making some assumptions...

1. That x and y are *orthogonal* to each other (point in completely different directions. 
2. That the dimensions are scaled equally (height and width are the same)

If we wanted to index the possible combinations by three different attributes, then we are in 3-D space and thus have a volume of space that points can fill.

For 4-D, we could organize all sets of 4 attribute positions into a tesseract.

![](https://upload.wikimedia.org/wikipedia/commons/5/55/8-cell-simple.gif)

We will keep things in 2-D for now, but the power of starting with a *space* is the we are explicit about our assumptions, including what are the dimensions.

Think about the space as what is possible.

# Vectors in space

So while space is all around in the background, we eventually what to differentiate information that occured or we are currently focusing on, from that backgound.

We do this with *vectors* in space. 

Now, it turns out, that if you have a space... you already know some special vectors. These are known as *basis vectors* and they define the space.

Lets draw a special vector that points in the x-direction (and only that direction) and has length 1. This is a path that starts at the origin, point `x=0, y=0` or `(0, 0)`, and ends at `x=1, y=0` or `(1, 0)` 

In [601]:
xUnitVector = pd.DataFrame({'x': [0., 1.], 'y': [0., 0.]})
yUnitVector = pd.DataFrame({'x': [0., 0.], 'y': [0., 1.]})
showX = alt.Chart(xUnitVector).mark_trail(color='red').encode(
    x='x:Q',
    y='y:Q',
    size='absX:Q').transform_calculate(absX=alt.expr.abs(alt.datum.x))
pointsIn2D + showX

alt.LayerChart(...)

I drew the thickness of the line segment proportional to the absolute length/magnitude of the vector.

Nows lets add a unit vector in the y-direction, from `(0, 0)` to `(0, 1)`.

In [602]:
showY = alt.Chart(yUnitVector).mark_trail(color='green').encode(
    x='x:Q',
    y='y:Q',
    size='absY:Q').transform_calculate(absY=alt.expr.abs(alt.datum.y))
pointsIn2D + showX + showY

alt.LayerChart(...)

You will notice that these look like axes. They point in the x and y direction (and we could flip them to go towards the negative numbers on each dimension). Let's do that.

In [603]:
showXN = alt.Chart(-xUnitVector).mark_trail(color='yellow').encode(
    x='x:Q',
    y='y:Q',
    size='absX:Q').transform_calculate(absX=alt.expr.abs(alt.datum.x))
showYN = alt.Chart(-yUnitVector).mark_trail(color='purple').encode(
    x='x:Q',
    y='y:Q',
    size='absY:Q').transform_calculate(absY=alt.expr.abs(alt.datum.y))
pointsIn2D + showX + showY + showXN + showYN

alt.LayerChart(...)

If we define the `(0, 1)` (red) vector as $i$, then the opposite vector `(0, -1)` (yellow) is simply $-1\times i$. 

There is nothing special about $-1$, this is what we call a *scalar*. A floating point number that *scales* vectors.

Lets try multiplying $i$ by some other scalars, where are two in two different colors

In [604]:
scalar = 2
show2 = alt.Chart(xUnitVector).mark_trail(color='white').encode(
    x='xS:Q',
    y='y:Q',
    size='absX:Q').transform_calculate(absX=alt.expr.abs(alt.datum.x),
                                      xS = scalar * alt.datum.x)

scalar = .5
showPoint5 = alt.Chart(xUnitVector).mark_trail(color='black').encode(
    x='xS:Q',
    y='y:Q',
    size='absX:Q').transform_calculate(absX=alt.expr.abs(alt.datum.x),
                                      xS = scalar * alt.datum.x)
pointsIn2D + show2 + showPoint5

alt.LayerChart(...)

So scalars increase or decrease the magnitude of the vector. 

You can test that out with the vector $i$ using the slider below.

In [605]:
slider = alt.binding_range(min=-5, max=5, step=.1)
selector = alt.selection_single(name="scalar", fields=["scalar"],
                               bind=slider, init={'scalar': 2})


popPop = alt.Chart(xUnitVector).mark_trail(color='red').encode(
    x='xS:Q',
    y='y:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                      xS = alt.datum.x * selector.scalar).add_selection(
    selector)

pointsIn2D + popPop

alt.LayerChart(...)

We can also add vectors to rotate around.

To see this, lets first define the y-direction vector, `(1, 0)` as $j$. 

We can, of course, scale this vector too.

Try it with the slider.

In [606]:
slider = alt.binding_range(min=-5, max=5, step=.1)
selector = alt.selection_single(name="scalar", fields=["scalar"],
                               bind=slider, init={'scalar': 2})


popPopY = alt.Chart(yUnitVector).mark_trail(color='green').encode(
    x='x:Q',
    y='yS:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                      yS = alt.datum.y * selector.scalar).add_selection(
    selector)

pointsIn2D + showX + popPopY

alt.LayerChart(...)

But most importantly, now that we have two vectors, we can add $i$ and $j$ so we get $i+j$.

Look at both here

In [607]:
pointsIn2D + showX + showY

alt.LayerChart(...)

To add vectors place them head to tail.

Order does not matter.

In this next plot, I shifted $j$ to the head of $i$

In [608]:
y_shifted = pd.DataFrame({'x': [1., 1.], 'y': [0., 1.]})

def vec_length():
    return alt.expr.sqrt(alt.datum.x + alt.datum.y)
showYShifted = alt.Chart(y_shifted).mark_trail(color='green').encode(
    x='x:Q',
    y='y:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y))
pointsIn2D + showX + showYShifted

alt.LayerChart(...)

$i+j$ gives us this purple vector `(1,1)`

In [609]:
x_plus_y = xUnitVector + yUnitVector
def vec_length():
    return alt.expr.sqrt(alt.datum.x + alt.datum.y)
vecXY = alt.Chart(x_plus_y).mark_trail(color='purple').encode(
    x='x:Q',
    y='y:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y))
pointsIn2D + showX + showYShifted + vecXY

alt.LayerChart(...)

We get the same vector if we shift $i$ to be at the head of $j$, see below.

In [610]:
x_shifted = pd.DataFrame({'x': [1., 0.], 'y': [1., 1.]})

def vec_length():
    return alt.expr.sqrt(alt.datum.x + alt.datum.y)
showXShifted = alt.Chart(x_shifted).mark_trail(color='red').encode(
    x='x:Q',
    y='y:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y))
pointsIn2D + showY + showXShifted

alt.LayerChart(...)

$j+i$ still gives us this purple vector `(1,1)`


In [611]:
pointsIn2D + showY + showXShifted + vecXY

alt.LayerChart(...)

Therefore,

If we have $v_{1} = (x_{1}, y_{1})$ and $v_{2} = (x_{2}, y_{2})$, then $v_{1}+v_{2} = (x_{1}+x_{2},~~ y_{1} + y_{2})$ 

You add the x-components of the vectors and the y-components of the vectors.

Now we can combine both scalars and our basis vectors to reach any point in the space!

Lets pick a point in the space and use $i$ and $j$ and 1 or 2 scalars to reach it.

Lets try, point $(2.5, 3)$.

In [612]:
pointA = pd.DataFrame({'x': [2.5], 'y': [3.]})
showA = alt.Chart(pointA).mark_circle(color='yellow', opacity=1, size=100).encode(
    x='x',
    y='y')
pointsIn2D + showA

alt.LayerChart(...)

Indeed, we can reach it:

This point will is reached by $2.5 \times i$ + $3 \times j$

In [613]:
scalar1 = 2.5
scalar2 = 3.
showS1 = alt.Chart(xUnitVector).mark_trail(color='red').encode(
    x='xS:Q',
    y='y:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                        xS=scalar1 * alt.datum.x)

showS2 = alt.Chart(yUnitVector).mark_trail(color='green').encode(
    x='x:Q',
    y='yS:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                        yS=scalar2* alt.datum.y)

pointsIn2D + showA + showS1 + showS2

alt.LayerChart(...)

In [614]:
showV3 = alt.Chart(xUnitVector + yUnitVector).mark_trail(color='purple').encode(
    x='xS:Q',
    y='yS:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                        xS=scalar1 * alt.datum.x,
                                        yS=scalar2 *alt.datum.y)

pointsIn2D + showA + showS1 + showS2 + showV3

alt.LayerChart(...)

Bingo.

This is important, because, it turns out $i$ and $j$, along with scalars can let us reach the whole space.

Lets explore the whole space just with these two basis vectors and 2 scalars.

You can play with the sliders below to see how multiplying scalars by each of the basis vectors allows us to reach points in the space.

In [615]:
slider1 = alt.binding_range(min=-5, max=5, step=.1)
slider2 = alt.binding_range(min=-5, max=5, step=.1)
selector1 = alt.selection_single(name="scalar1", fields=['scalar1'],
                                bind=slider1, init={'scalar1': 2.5})
selector2 = alt.selection_single(name="scalar2", fields=['scalar2'],
                                bind=slider2, init={'scalar2': 3})


alt.layer(
    pointsIn2D,
    alt.Chart(pd.DataFrame({'x': [0], 'y': [0]})).mark_circle(color='yellow', opacity=1, size=100).encode(
    x='xS:Q',
    y='yS:Q').transform_calculate(xS = selector1.scalar1,
                                      yS = selector2.scalar2),
    alt.Chart(xUnitVector).mark_trail(color='red').encode(
    x='xS:Q',
    y='yS:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                      xS = alt.datum.x * selector1.scalar1,
                                      yS = alt.datum.y * selector2.scalar2),
    alt.Chart(yUnitVector).mark_trail(color='green').encode(
    x='xS:Q',
    y='yS:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                      xS = alt.datum.x * selector1.scalar1,
                                      yS = alt.datum.y * selector2.scalar2),
    alt.Chart(xUnitVector + yUnitVector).mark_trail(color='purple').encode(
    x='vxS:Q',
    y='vyS:Q',
    size='length:Q').transform_calculate(length=alt.expr.sqrt(alt.datum.x + alt.datum.y),
                                      vxS = alt.datum.x * selector1.scalar1,
                                      vyS = alt.datum.y * selector2.scalar2)
).add_selection(
    selector1, selector2)

alt.LayerChart(...)

Vectors seem fundamental.

The rows of your data can be thought of as vectors, for example.

But remember that vectors are embedded in a space. Quite literally, every vector is defined by the axes/basis vectors of the space. 

# Enter (Then Exit) the Matrix

It is that idea of space that is fundamental. To see that more clearly, we need to move/jostle the space. 

Matrices in common parlance are actually two different concepts, and we need to unpack them.

## static matrix storage
In the first case, your data might be thought of as a stacking of vectors. If there were two variables, we could plot all of the instances of your data as the line-segments/points that they point to. 

Something like this:

In [616]:
SEED = 412412 
N= 100 # number of rows to generate
M = 2 # number of columns to generate
MIN = -5
MAX = 5
rngType = np.random._generator.Generator
def generate_bounded_array(out_shape: Tuple, rng_instance: rngType, min: float=-1, max: float=1) -> np.ndarray:
    """Generate a uniform random np.ndarray of size n_size where values are bounded
    Paramaters
     out_shape
     rng_instance
     min (default -1)
     max (default 1)
    Returns
     np.ndarray of size n_size
    """
    return rng_instance.random(out_shape)*(max - min) + min
my_rng = np.random.default_rng(SEED)
my_matrix = generate_bounded_array((N,M), my_rng, MIN, MAX)
my_matrix[0:10, :]

array([[-0.30534974,  4.75123598],
       [ 1.40944216,  1.00196978],
       [-1.14133491, -0.17873674],
       [ 2.3235587 ,  0.21451174],
       [-4.7626046 ,  2.69753341],
       [-2.24395023, -1.3983755 ],
       [ 1.05021482,  0.40494259],
       [-4.1045666 ,  3.88056775],
       [ 4.87016982, -1.15889813],
       [-2.74087378, -3.41316753]])

The `my_matrix` object is a numpy ndarray technically, you can see it as a stacking of 2-D vectors. Where the first coordinate is in the first column, and the second coordinate is in the second column, and each row holds a vector.

I generated random numbers from (0, 1) from `np.random.default_rng` while setting the seed so that the numbers are replicable. See [here](https://albertcthomas.github.io/good-practices-random-number-generators/) for more on using numpy to generate random numbers is a principled way.

Lets visualization this stacking of vectors.

In [617]:
my_data = pd.DataFrame({'originX': [0.0 for i in range(N)],
                       'originY': [0.0 for i in range(N)],
                       'x2': list(my_matrix[:, 0]),
                       'y2': list(my_matrix[:, 1])})

In [618]:
dataVec = alt.Chart(my_data).mark_rule().encode(
    x='originX:Q',
    y='originY:Q',
    x2='x2:Q',
    y2='y2:Q')

dataPoints = alt.Chart(my_data).mark_circle(opacity=.5, color='black').encode(
    x='x2:Q',
    y='y2:Q')

pointsIn2D + dataVec + dataPoints

alt.LayerChart(...)

Whether you think about your data as points, line segments, arrows, etc, they are static in this representation. They just sit there.

Side note: I am going to switch to `mark_rule` from here on out because it is a bit easier to work with. 

Often, this stacking of vectors of the same length will be called a *matrix*. Where a vector is defined by the dimensionaity of the space it sits within (for example in 2-space vectors have 2 values), a matrix is defined by its columns and rows. Therefore the `my_matrix` object has a shape attribute that holds two values: the rows numbers and the column numbers.

In [619]:
my_matrix.shape

(100, 2)

We can stack 2-D vectors in rows, like in the `my_matrix` DataFrame, or we can stack them in columns, which would be the transpose of that; where rows become columns and vice versa.

Here are the first 10 data vectors (each of length 2) transposed.

In [620]:
print(my_matrix.transpose().shape)
my_matrix.transpose()[:, 0:10]

(2, 100)


array([[-0.30534974,  1.40944216, -1.14133491,  2.3235587 , -4.7626046 ,
        -2.24395023,  1.05021482, -4.1045666 ,  4.87016982, -2.74087378],
       [ 4.75123598,  1.00196978, -0.17873674,  0.21451174,  2.69753341,
        -1.3983755 ,  0.40494259,  3.88056775, -1.15889813, -3.41316753]])

Again, we are just storing the same information statically.

But matrices are so much more than this.

## active matrix processing
 
It is equally, or perhaps even more important, to think of (and use)  matrices as *transformers*. They take input and produce output. 

In fact, matrices are special transformers, they make a *linear* transformation from input to output.

This involves adding and multiplying values element-wise -- the computation is the uncool part.

The cool part is that matrices can transform not just vectors, and points, but the space itself (I know... awesome).

We will look at transformations of the form:

$$
\underbrace{M}_{m\times p} \overbrace{V'}^{p \times n}
$$

 We will start with a static matrix $V$ that has $n$ vectors in $p$-space.
 
 We are taking the transpose of this, that is what $V'$ is $p \times n$.
 
 The way that matrix right-multiplication works is that we are going to take each row of $M$ (of which there are p-values across the columns), and do element-by-element multiply them by each column of $V'$ (of which there are p-values across the rows), and then sum those products.
 
 The resulting matrix $G$ will have dimensions of $m \times n$ where the each element, $ij$, is the result of multiplying and summing the $i$th row in $M$ with the $j$th column in $V'$.  
 
 Take the simple case where we begin with our two basis vectors 
 
 $$
 V = 
 \left[
 \begin{array}{cc}
 1 & 0 \\
 0 & 1 \\
 \end{array}
 \right]
 $$
 
 
with $x$(top row) then $y$(bottom row), then in this case:
 
$$
 V' = 
 \left[
 \begin{array}{cc}
 1 & 0 \\
 0 & 1 \\
 \end{array}
 \right]
 $$
 
 But now $x$ is down the left column and y is down the right column.
 
 Define
 
 $$
 M = 
 \left[
 \begin{array}{cc}
 .5 & 2 \\
 .2 & .5 \\
 \end{array}
 \right]
 $$
 
 We know that the result is going to be  $2 \times 2$ output (we can think of this as a static matric of output vectors in a transformed space).
 
- The upper, left-most value will be the sum of the elementwise products of the **first** row of $M$ times the **first** column of $V'$, which is $(.5 \times 1) + (2 \times 0) = .5 $

- The upper, right-most value will be the sum of the elementwise products of the **first** row of $M$ times the **second** column of $V'$, which is $(.5 \times 0) + (2 \times 1) = 2 $

- The lower, right-most value will be the sum of the elementwise products of the **second** row of $M$ times the **first** column of $V'$, which is $(.2 \times 1) + (.5 \times 0) = .2 $

- The lower, left-most value will be the sum of the elementwise products of the **second** row of $M$ times the **second** column of $V'$, which is $(.2 \times 0) + (.5 \times 1) = .5 $


 
 Then we have
 
 $$
 MV' = 
 \left[
 \begin{array}{cc}
 .5 & 2 \\
 .2 & .5 \\
 \end{array}
 \right]
 \left[
 \begin{array}{cc}
 1 & 0 \\
 0 & 1 \\
 \end{array}
 \right]
 = 
 \left[
 \begin{array}{cc}
 .5 & 2 \\
 .2 & .5 \\
 \end{array}
 \right]
 $$
 
 
 This might seem un-inspiring but it is actually really important. This tells us that the rows of the matrix $M$ are the coordinates/vectors in the output space that correspond to the unit x and y basis vectors. That is cool.
 
 
Our computer can do linear algebra too, and that is a really improtant use of your computational companion.

numpy is terrific for these types of operations. We can use the `@` for matrix multiplication.

In addition the `np.array.transpose` method is available to transpose the input. 

Note that matrix multiplication is different to the right and to the left. You should try some examples out.

In [621]:
V = np.array([[1., 0.],
              [0., 1.]])
M = np.array([[.5, 2.],
              [.2, .5]])
G = M@V.transpose()
print(f"The answer is\n {G}")

The answer is
 [[0.5 2. ]
 [0.2 0.5]]


Your computer is pretty good at this stuff, so lets make sure you are up to speed too.

We can visualization this transformation:

In [622]:


basisVecs = pd.DataFrame({'xB': V[0], 'yB': V[1]})

TransformedVecs = pd.DataFrame({'xB': V[0], 
                                'yB': V[1],
                                'xT': G[0],
                                'yT': G[1]})

showBasis = alt.Chart(basisVecs).mark_circle(size=40).encode(
    x='xB',
    y='yB')

showTransformedLine = alt.Chart(TransformedVecs).mark_rule(size=2, color='red').encode(
    x='xB',
    y='yB',
    x2='xT',
    y2='yT')

showTransformedPoint = alt.Chart(TransformedVecs).mark_circle(size=100, color='red').encode(
    x='xT',
    y='yT')

pointsIn2D  + showBasis + showTransformedLine + showTransformedPoint

alt.LayerChart(...)

The filled in blue dots are where $i$ and $j$ the original basis vectors started, as stacked in $V$.

The larger red dots are where the transformation $M$ takes them. 

The red line traces the path from the old point to the new point.

More broadly, this matrix $M$ transforms every possible vector in this space, not just the basis vectors $i$ and $j$.

Lets transform the data points in `my_data`, this is as simple as slotting in the data vectors for $V'$

- We will create a new object for the transformed vectors, `G_data`.

- then plot the:
   - original points
   - transformed points
   - connect them with a line

In [623]:
G_data = M@my_data[['x2', 'y2' ]].to_numpy().transpose()

In [624]:
my_data['xT'], my_data['yT'] = G_data[0], G_data[1]

base = alt.Chart(my_data)

dataPoints = base.mark_circle(opacity=.5, color='black').encode(
    x='x2:Q',
    y='y2:Q')

dataPointsTransformedLine = base.mark_rule(color='red').encode(
    x='x2:Q',
    y='y2:Q',
    x2='xT:Q',
    y2='yT:Q')

dataPointsTransformedPoints = base.mark_circle(size=100, color='red').encode(
    x='xT:Q',
    y='yT:Q')


dataPoints + dataPointsTransformedLine + dataPointsTransformedPoints

alt.LayerChart(...)

This gives us some sense that the points are being turned and shifted.


Below you can play with the 4 values in this $M$ to see how it changes the output of the transformation.

This should give you a better sense of how a matrix can transform space, even of the same dimensionality (here 2 dimensions)


In [625]:
slider00 = alt.binding_range(min=-2, max=2, step=.1)
slider01 = alt.binding_range(min=-2, max=2, step=.1)
slider10 = alt.binding_range(min=-2, max=2, step=.1)
slider11 = alt.binding_range(min=-2, max=2, step=.1)
selector00 = alt.selection_single(name="val00", fields=['val00'],
                                bind=slider00, init={'val00': M[0,0]})
selector01 = alt.selection_single(name="val01", fields=['val01'],
                                bind=slider01, init={'val01': M[0, 1]})
selector10 = alt.selection_single(name="val10", fields=['val10'],
                                bind=slider10, init={'scalar10': M[1, 0]})
selector11 = alt.selection_single(name="val11", fields=['val11'],
                                bind=slider22, init={'val11': M[1, 1]})


dataPointsTransformedLine = base.mark_rule(opacity=.5, color='red').encode(
    x='x2:Q',
    y='y2:Q',
    x2='xT22:Q',
    y2='yT22:Q').transform_calculate(xT22= selector11.val11 * alt.datum.x2 + selector12.val12 * alt.datum.y2,
                                   yT22= selector21.val21 * alt.datum.x2 + selector22.val22 * alt.datum.y2)



dataPointsTransformedPoints = alt.Chart(my_data).mark_circle(size=100, color='red').encode(
    alt.X('xT2:Q', scale=alt.Scale(
            domain=(-20, 20),
            clamp=True
        )),
    alt.Y('yT2:Q', scale=alt.Scale(
            domain=(-20, 20),
            clamp=True
        ))).transform_calculate(xT2= selector11.val11 * alt.datum.x2 + selector12.val12 * alt.datum.y2,
                                   yT2= selector21.val21 * alt.datum.x2 + selector22.val22 * alt.datum.y2)

alt.layer(
    dataPoints,
    dataPointsTransformedLine,
    dataPointsTransformedPoints).add_selection(selector11, selector12, selector21, selector22).interactive()



alt.LayerChart(...)

In the above, make sure you play with each range setting the other ranges to different settings.

It is important to try and visualize the matrix transformation applying forces to the original points and moving them. It is *processing* them. This is more than static, it is active.

There is nothing special about this particular $M$. The fact that $M$ was a $2 \times 2$ matrix meant that we were mapping an input $V$ in 2-D back to 2-D.

But we could also move data to other spaces! 

If $M$ was $1 \times 2$ we would be mapping a 2-D V into a 1 dimensional space; that just means projecting down to a line. 

If $M$ was $3 \times 2$ we would be mapping a 2-D V into a 3-dimensional space. 

And we could go on to any dimensions for both input and output.

That is why matrices are so powerful. 

Remember that:

- the space is made up of all the potential vectors and points

- matrices linearly transform vectors into new vectors

- that means that matrices transform the space within which vectors live!

That is pretty important, transforming the space.

These are the ideas that animate dimensionality reduction techniques in particular, but are also related to regression and regularization.